In [0]:
import os
import logging
from pyspark.sql import SparkSession, functions as F
from pyspark.sql.window import Window
from pyspark.sql.functions import col
from datetime import datetime

spark = SparkSession.builder.appName("Average_Failures_by_EquipmentGroup").getOrCreate()
timestamp = datetime.now().strftime("%Y-%m-%d_%H-%M-%S")

logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')
logger = logging.getLogger("Average_Failures_by_EquipmentGroup")
logger.info(f"{timestamp} [INFO] Inciando Transformação de dados: Average Failures by Equipment Group") 

2024-04-27 00:21:57,227 - INFO - 2024-04-27_00-21-57 [INFO] Inciando Transformação de dados: Average Failures by Equipment Group


In [0]:
logger.info(f"{timestamp} [INFO] Carregando tabelas: 'equpment_failure_sensors_loaded'; 'df_sensors_loaded'; 'equipment_loaded'") 
df_logs = spark.read.parquet("dbfs:/FileStore/workflow/equpment_failure_sensors_loaded.parquet")
df_sensors = spark.read.parquet("dbfs:/FileStore/workflow/df_sensors_loaded.parquet")
df_equipment = spark.read.parquet("dbfs:/FileStore/workflow/equipment_loaded.parquet")

df_logs_alias = df_logs.alias("logs")
df_sensors_alias = df_sensors.alias("sensors")
df_equipment_alias = df_equipment.alias("equipment")
logger.info(f"{timestamp} [INFO] Ação realizada com sucesso!")

2024-04-27 00:21:57,342 - INFO - 2024-04-27_00-21-57 [INFO] Carregando tabelas: 'equpment_failure_sensors_loaded'; 'df_sensors_loaded'; 'equipment_loaded'
2024-04-27 00:21:59,081 - INFO - 2024-04-27_00-21-57 [INFO] Ação realizada com sucesso!


In [0]:
# Criação de dataframes
try: 
    logger.info(f"{timestamp} [INFO] Criando DataFrame...")
    failures_with_equipment = df_logs_alias.join(
        df_sensors_alias,
        df_logs_alias["sensor_id"] == df_sensors_alias["sensor_id"],
        "inner"
    ).join(
        df_equipment_alias,
        df_sensors_alias["equipment_id"] == df_equipment_alias["equipment_id"],
        "inner"
    )
except Exception as e:
    logger.error(f"{timestamp} [ERROR] Falha na criação do Dataframe: {e}.")

if 'group_name' not in failures_with_equipment.columns:
    logger.error(f"{timestamp} [ERROR] Coluna 'group_name' não encontrada em df_equipment_alias.")
else:
    logger.info(f"{timestamp} [INFO] DataFrame criado com sucesso, contendo a coluna 'group_name'.")
    logger.info(f"{timestamp} [INFO] Ação realizada com sucesso!")

try: 
    # Atualizando dataframes
    logger.info(f"{timestamp} [INFO] Criando DataFrame...")
    failures_with_equipment = failures_with_equipment.select(
        df_equipment_alias["group_name"],
        df_equipment_alias["equipment_id"],
        df_logs_alias["status"]
    )
    if 'status' not in failures_with_equipment.columns:
        logger.error(f"{timestamp} [ERROR] Coluna 'status' não encontrada em df_equipment_alias.")
    else:
        logger.info(f"{timestamp} [INFO] DataFrame criado com sucesso, contendo a coluna 'status'.")
        logger.info(f"{timestamp} [INFO] Ação realizada com sucesso!")
except Exception as e:
    logger.error(f"{timestamp} [ERROR] Falha na criação do Dataframe: {e}.")


2024-04-27 00:21:59,149 - INFO - 2024-04-27_00-21-57 [INFO] Criando DataFrame...
2024-04-27 00:21:59,168 - INFO - 2024-04-27_00-21-57 [INFO] DataFrame criado com sucesso, contendo a coluna 'group_name'.
2024-04-27 00:21:59,170 - INFO - 2024-04-27_00-21-57 [INFO] Ação realizada com sucesso!
2024-04-27 00:21:59,171 - INFO - 2024-04-27_00-21-57 [INFO] Criando DataFrame...
2024-04-27 00:21:59,186 - INFO - 2024-04-27_00-21-57 [INFO] DataFrame criado com sucesso, contendo a coluna 'status'.
2024-04-27 00:21:59,187 - INFO - 2024-04-27_00-21-57 [INFO] Ação realizada com sucesso!


In [0]:
# Calculo da media de falhas por Grupo de Equipamento 
logger.info(f"{timestamp} [INFO] Inciando o calculo da media de falhas por Grupo de Equipamento") 
equipment_failure_counts = failures_with_equipment.filter(failures_with_equipment['status'] == 'ERROR').groupBy(
    "group_name"
).agg(
    F.count("equipment_id").alias("equipment_failure_count")
)

# Cálculo do total de falhas em todos os grupos
total_failures = equipment_failure_counts.select(
    F.sum("equipment_failure_count").alias("total_failures")
).collect()[0]["total_failures"]

# Adição da coluna de porcentagem
equipment_failure_counts = equipment_failure_counts.withColumn(
    "percentage_of_total",
    (F.col("equipment_failure_count") / total_failures) * 100
)

# Ordenação das médias de falhas por grupo em ordem crescente
avg_failures_per_group = equipment_failure_counts.groupBy("group_name").agg(
    F.avg("equipment_failure_count").alias("avg_failures"),
    F.avg("percentage_of_total").alias("avg_percentage_of_total")
).orderBy("avg_failures")

logger.info(f"{timestamp} [INFO] Média de falhas por grupo de equipamento calculada e ordenada com percentuais.")
logger.info(f"{timestamp} [INFO] Ação realizada com sucesso!")

# Exibindo os resultados
display(avg_failures_per_group)


2024-04-27 00:21:59,246 - INFO - 2024-04-27_00-21-57 [INFO] Inciando o calculo da media de falhas por Grupo de Equipamento
2024-04-27 00:22:05,777 - INFO - 2024-04-27_00-21-57 [INFO] Média de falhas por grupo de equipamento calculada e ordenada com percentuais.
2024-04-27 00:22:05,778 - INFO - 2024-04-27_00-21-57 [INFO] Ação realizada com sucesso!


group_name avg_failures avg_percentage_of_total Z9K1SAP4 340549.0 7.170245132356734 PA92NCXZ 678141.0 14.278230751819937 9N127Z5P 680731.0 14.332763094868382 NQWPA8D3 681636.0 14.351817832497277 VAPQY59S 1023549.0 21.55078192852894 FGHQWR2Q 1344869.0 28.31616125992873

In [0]:
# Salvando a tabela no databricks
logger.info(f"{timestamp} [INFO] Criando tabelas: 'avg_failures_per_group'...") 
avg_failures_per_group.write.option("overwriteSchema", "true").format("delta").mode("overwrite").saveAsTable("avg_failures_per_group")
display(avg_failures_per_group)
logger.info(f"{timestamp} [INFO] Ação realizada com sucesso!")
logger.info(f"{timestamp} [INFO] Average Failures by Equipment Group - Finalizado com sucesso!") 

2024-04-27 00:22:11,362 - INFO - 2024-04-27_00-21-57 [INFO] Criando tabelas: 'avg_failures_per_group'...


group_name avg_failures avg_percentage_of_total Z9K1SAP4 340549.0 7.170245132356734 PA92NCXZ 678141.0 14.278230751819937 9N127Z5P 680731.0 14.332763094868382 NQWPA8D3 681636.0 14.351817832497277 VAPQY59S 1023549.0 21.55078192852894 FGHQWR2Q 1344869.0 28.31616125992873

2024-04-27 00:22:23,250 - INFO - 2024-04-27_00-21-57 [INFO] Ação realizada com sucesso!
2024-04-27 00:22:23,252 - INFO - 2024-04-27_00-21-57 [INFO] Average Failures by Equipment Group - Finalizado com sucesso!
